In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import sys
sys.path.append('../src')

In [ ]:
from d00_utils.db_utils import dbReadWriteClean, dbReadWriteViews
io_clean = dbReadWriteClean()
io_views = dbReadWriteViews()

In [ ]:
measurement_abstract_rpt_df = io_clean.get_table("measurement_abstract_rpt")
print(measurement_abstract_rpt_df.shape)
measurement_abstract_rpt_df.head()

In [ ]:
instances_w_labels_test_df = io_views.get_table('instances_w_labels_test')
instances_w_labels_test_df = instances_w_labels_test_df[(instances_w_labels_test_df['view']=='a2c') | (instances_w_labels_test_df['view']=='a4c')]
print(instances_w_labels_test_df.shape)
instances_w_labels_test_df.head()

In [ ]:
instances_w_labels_test_downsampleby5_df = io_views.get_table('instances_w_labels_test_downsampleby5')
instances_w_labels_test_downsampleby5_df = instances_w_labels_test_downsampleby5_df[(instances_w_labels_test_downsampleby5_df['view']=='a2c') | (instances_w_labels_test_downsampleby5_df['view']=='a4c')]
print(instances_w_labels_test_downsampleby5_df.shape)
instances_w_labels_test_downsampleby5_df.head()

In [ ]:
a_measgraphref_df = io_clean.get_table("a_measgraphref")
print(a_measgraphref_df.shape)
a_measgraphref_df.head()

In [ ]:
def get_recommendation(row):
    return 'normal' if row['ef'] >= 60 else 'abnormal' if row['ef'] < 40 else 'greyzone'

In [ ]:
merge_df = measurement_abstract_rpt_df[['studyidk', 'measabstractnumber', 'name']]
merge_df = merge_df.merge(a_measgraphref_df, on=['studyidk', 'measabstractnumber'])
merge_df = merge_df[['studyidk', 'instanceidk', 'indexinmglist', 'meassequence', 'name', 'value', 'howentered']]
print(merge_df.shape)
merge_df.head()

In [ ]:
start_df = merge_df
filter_df = start_df

In [ ]:
filter_df = filter_df[filter_df['name'].str.contains('ps4')]
print(len(start_df))
print(len(filter_df))
print(len(filter_df)/len(start_df))

In [ ]:
filter_df['name'].value_counts()

In [ ]:
filter_df = filter_df[filter_df['name'].isin(['VTD(MDD-ps4)', 'VTS(MDD-ps4)'])]
filter_df = filter_df[filter_df['value']>0]
print(len(start_df))
print(len(filter_df))
print(len(filter_df)/len(start_df))

In [ ]:
filter_df.head()

In [ ]:
diastole_df = filter_df[filter_df['name'].str.contains('VTD')]
print(diastole_df.shape)
diastole_df.head()

In [ ]:
systole_df = filter_df[filter_df['name'].str.contains('VTS')]
print(systole_df.shape)
systole_df.head()

In [ ]:
diastole_df['value'].describe()

In [ ]:
plt.title(f'Count of Measured End-Diastole Volume Values')
plt.xlabel('Value')
plt.ylabel('Counts')
plt.hist(diastole_df['value'], bins=20);

In [ ]:
systole_df['value'].describe()

In [ ]:
plt.title(f'Count of Measured End-Systole Volume Values')
plt.xlabel('Value')
plt.ylabel('Counts')
plt.hist(systole_df['value'], bins=20);

In [ ]:
agg_diastole_df = diastole_df
agg_diastole_df = agg_diastole_df.groupby(['studyidk', 'instanceidk', 'indexinmglist']).agg({"name": pd.Series.unique, "value": pd.Series.median}).reset_index()
print(agg_diastole_df.shape)
agg_diastole_df.head()

In [ ]:
agg_diastole_df = agg_diastole_df.groupby(['studyidk', 'instanceidk']).agg({"name": pd.Series.unique, "value": pd.Series.median}).reset_index()
print(agg_diastole_df.shape)
agg_diastole_df.head()

In [ ]:
agg_systole_df = systole_df
agg_systole_df = agg_systole_df.groupby(['studyidk', 'instanceidk', 'indexinmglist']).agg({"name": pd.Series.unique, "value": pd.Series.median}).reset_index()
print(agg_systole_df.shape)
agg_systole_df.head()

In [ ]:
agg_systole_df = agg_systole_df.groupby(['studyidk', 'instanceidk']).agg({"name": pd.Series.unique, "value": pd.Series.median}).reset_index()
print(agg_systole_df.shape)
agg_systole_df.head()

In [ ]:
both_df = agg_diastole_df.merge(agg_systole_df, on=['studyidk', 'instanceidk'], suffixes=['_diastole', '_systole'])
both_df = both_df[both_df['value_diastole']>=both_df['value_systole']]
print(both_df.shape)
both_df.head()

In [ ]:
both_df['ef'] = (both_df['value_diastole']-both_df['value_systole'])/both_df['value_diastole']*100

In [ ]:
both_df['ef'].describe()

In [ ]:
plt.title(f'Count of All Measured Ejection Fraction Values')
plt.xlabel('Value')
plt.ylabel('Counts')
plt.hist(both_df['ef'], bins=20);

In [ ]:
both_df['recommendation'] = both_df.apply(get_recommendation, axis=1)

In [ ]:
both_df['recommendation'].value_counts()

In [ ]:
both_df['recommendation'].value_counts()/both_df['recommendation'].value_counts().sum()

In [ ]:
start_df = both_df
filter_df = start_df

In [ ]:
# filter_df = filter_df[filter_df['instanceidk'].isin(instances_w_labels_test_df['instanceidk'].unique())]
# print(start_df.shape)
# print(filter_df.shape)
# print(len(filter_df)/len(start_df))

In [ ]:
filter_df = filter_df[filter_df['instanceidk'].isin(instances_w_labels_test_downsampleby5_df['instanceidk'].unique())]
print(start_df.shape)
print(filter_df.shape)
print(len(filter_df)/len(start_df))

In [ ]:
filter_df = filter_df[filter_df['instanceidk'].isin(a_measgraphref_df['instanceidk'].unique())]
print(start_df.shape)
print(filter_df.shape)
print(len(filter_df)/len(start_df))

In [ ]:
len(filter_df['studyidk'].unique())

In [ ]:
len(filter_df['instanceidk'].unique())

In [ ]:
filter_df['value_diastole'].describe()

In [ ]:
plt.title(f'Count of Filtered End-Diastole Volume Values')
plt.xlabel('Value')
plt.ylabel('Counts')
plt.hist(filter_df['value_diastole'], bins=20);

In [ ]:
filter_df['value_systole'].describe()

In [ ]:
plt.title(f'Count of Filtered End-Systole Volume Values')
plt.xlabel('Value')
plt.ylabel('Counts')
plt.hist(filter_df['value_systole'], bins=20);

In [ ]:
filter_df['ef'].describe()

In [ ]:
plt.title(f'Count of Filtered Measured Ejection Fraction Values')
plt.xlabel('Value')
plt.ylabel('Counts')
plt.hist(filter_df['ef'], bins=20);

In [ ]:
filter_df['recommendation'].value_counts()

In [ ]:
filter_df['recommendation'].value_counts()/filter_df['recommendation'].value_counts().sum()

In [ ]:
filter_df.head()